In [1]:
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
from model import create_model
from align import AlignDlib
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import sys
from PIL import Image
sys.modules['Image'] = Image
from PIL import Image
import warnings
from scipy import ndimage
warnings.filterwarnings('ignore')

In [3]:
# Model for 128 embeddings from a image
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

In [4]:
# Model Summary
nn4_small2_pretrained.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 102, 102, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 48, 48, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [12]:
# Initialize the OpenFace face alignment utility
alignment = AlignDlib('weights/landmarks.dat')

In [13]:
# Funtion to align an image using dlib utility
def align_image(img):
    return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), 
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

In [14]:
def load_image(path):
    img = cv2.imread(path, 1)
    return img[...,::-1]

In [15]:
class IdentityMetadata():
    def __init__(self, base, name, file):
        self.base = base
        self.name = name
        self.file = file

    def __repr__(self):
        return self.image_path()
    
    def subdir_path(self):
        return self.name
    
    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
    
def load_metadata(path):
    metadata = []
    for i in sorted(os.listdir(path)):
        for f in sorted(os.listdir(os.path.join(path, i))):
            # Check file extension. Allow only jpg/jpeg' files.
            ext = os.path.splitext(f)[1]
            if ext == '.jpg' or ext == '.jpeg':
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

metadata = load_metadata('images')

In [16]:
datagen = ImageDataGenerator(rotation_range=15, 
                             width_shift_range=0.1, 
                             height_shift_range=0.1,
                             shear_range=0.15, 
                             zoom_range=0.1,
                             channel_shift_range = 10, 
                             brightness_range=[0.2,1.0],
                             horizontal_flip=True)
import os

folder_name='AugmentedImages/'
save_here = folder_name

if not os.path.exists(save_here):
    os.makedirs(save_here) 

    for i, m in enumerate(metadata): 
        img = load_image(m.image_path())
        datagen.fit(np.expand_dims(img,0))
        directory=save_here+m.subdir_path()
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(directory)
        for x, val in zip(datagen.flow(np.expand_dims(img,0), 
                                       save_to_dir=directory, 
                                       save_prefix='AugImage', 
                                       save_format='jpg'),range(3)):
            pass

AugmentedImages/Aastha
AugmentedImages/AbhishekRanjan
AugmentedImages/Unknown


In [17]:
metadata1 = load_metadata('AugmentedImages')

In [18]:
# To store image embeddings
embedded = np.zeros((metadata1.shape[0], 128))

# Process image through the neural net to get 128 embeddings per image
for i, m in enumerate(metadata1):    
    print(i, " ",m)
    img = load_image(m.image_path())
    img = align_image(img)
    if img is None:
        print("No face found", m)
        continue
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    # obtain embedding vector for image
    embedded[i] = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]

0   AugmentedImages\Aastha\AugImage_0_1336.jpg
1   AugmentedImages\Aastha\AugImage_0_1410.jpg
2   AugmentedImages\Aastha\AugImage_0_1495.jpg
3   AugmentedImages\Aastha\AugImage_0_1900.jpg
4   AugmentedImages\Aastha\AugImage_0_2015.jpg
5   AugmentedImages\Aastha\AugImage_0_2054.jpg
6   AugmentedImages\Aastha\AugImage_0_2076.jpg
7   AugmentedImages\Aastha\AugImage_0_2396.jpg
8   AugmentedImages\Aastha\AugImage_0_2437.jpg
9   AugmentedImages\Aastha\AugImage_0_2470.jpg
10   AugmentedImages\Aastha\AugImage_0_2483.jpg
11   AugmentedImages\Aastha\AugImage_0_2562.jpg
12   AugmentedImages\Aastha\AugImage_0_262.jpg
13   AugmentedImages\Aastha\AugImage_0_2633.jpg
14   AugmentedImages\Aastha\AugImage_0_2826.jpg
15   AugmentedImages\Aastha\AugImage_0_3214.jpg
16   AugmentedImages\Aastha\AugImage_0_3344.jpg
17   AugmentedImages\Aastha\AugImage_0_3568.jpg
18   AugmentedImages\Aastha\AugImage_0_3921.jpg
19   AugmentedImages\Aastha\AugImage_0_4147.jpg
20   AugmentedImages\Aastha\AugImage_0_4179.jpg
21 

In [19]:
from sklearn.preprocessing import LabelEncoder
targets = np.array([m.name for m in metadata1])
encoder = LabelEncoder()
encoder.fit(targets)

# Numerical encoding of identities
y = encoder.transform(targets)

In [20]:
train_idx = np.arange(metadata1.shape[0]) % 2 != 0
test_idx = np.arange(metadata1.shape[0]) % 2 == 0

# 50 train examples of 10 identities (5 examples each)
X_train = embedded[train_idx]
# 50 test examples of 10 identities (5 examples each)
X_test = embedded[test_idx]

y_train = y[train_idx]
y_test = y[test_idx]

In [21]:
### SUPPORT VECTOR CLASSIFIER
# svc = SVC(kernel="linear", C=0.025) #10 -> 57% accuracy, 30-> 31%, 50->45%
# svc = SVC(gamma=2, C=1, probability=True) #30-> 88%, 50->88%

### Gaussian CLASSIFIER
# svc = GaussianProcessClassifier(1.0 * RBF(1.0)) # 30->90%, 50->89%

### KNN CLASSIFIER
# svc = KNeighborsClassifier(2) #10 images -> 85% accuracy, 30,50 images->88%
# svc = KNeighborsClassifier(5) #10 -> 78% accuracy, 30-> 86%, 50->87%

### DECISION TREE CLASSIFIER
# svc = DecisionTreeClassifier(max_depth=5) # 30,50->58%
# svc = DecisionTreeClassifier(max_depth=10) # 30->68%, 50->79%
# svc = DecisionTreeClassifier(max_depth=20) #30-> 70%, 50->80%
# svc = DecisionTreeClassifier(max_depth=30) # 30->73%, 50->80%
# svc = DecisionTreeClassifier(max_depth=40) # 30->73%, 50->78%

### Random Forest Classifier
# svc = RandomForestClassifier(max_depth=10, n_estimators=50, max_features=1)  #10 -> 80% accuracy aprox, 30,50 -> 88%
# svc = RandomForestClassifier(40)  #10 -> 80% accuracy aprox, 30,50 -> 89%
# svc = RandomForestClassifier(30)  #10 -> 70% accuracy aprox, 30,50 -> 89%
# svc = RandomForestClassifier(20)  #10 -> 65% accuracy aprox, 30,50 -> 87%

### MLP CLASSIFIER
svc = MLPClassifier(alpha=1.0, max_iter=1000) # 30,50->88%

### ADA BOOST CLASSIFIER
# svc = AdaBoostClassifier(n_estimators=100, random_state=0) # 30->35%, 50->28%

### QUADRATIC CLASSIFIER
# svc = QuadraticDiscriminantAnalysis() #30->40%, 50->9%

#Naive bayes algorithm
# svc = GaussianNB() #10 -> 73% accuracy aprox, 30 -> 86%, 50 -> 88%

svc.fit(X_train, y_train)

acc_svc = accuracy_score(y_test, svc.predict(X_test))

print(f'SVM accuracy = {acc_svc}')

SVM accuracy = 1.0


In [22]:
# train_idx = np.arange(metadata.shape[0]) 

# # 50 train examples of 10 identities (5 examples each)
# X_train = embedded[train_idx]

# y_train = y[train_idx]

# # svc = KNeighborsClassifier()
# # svc = SVC()
# svc = RandomForestClassifier(20)
# svc.fit(X_train, y_train)


In [23]:
# Prediction Code for live Images

In [24]:
font = cv2.FONT_HERSHEY_SIMPLEX

def processFrame(img):
    # Change RGB to BGR for imshow
    origImg = img[...,::-1]
    
    # Get bounding box
    bb = alignment.getLargestFaceBoundingBox(img)
    
    # Align image
    img = align_image(img)
    
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    
    # obtain embedding vector for image
    features = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
    
    # Get the class for prediction
    predClass = svc.predict([features])
    predClass = encoder.inverse_transform(predClass)[0]
    
    # Display the image with bounding box & label
    cv2.putText(origImg,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
    cv2.rectangle(origImg,(bb.left(),bb.top()),(bb.right(),bb.bottom()),(0,0,255),2)
    cv2.imshow('image',origImg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [25]:
font = cv2.FONT_HERSHEY_SIMPLEX

def processFrame1(img):
    # Change RGB to BGR for imshow
    origImg = img[...,::-1]
    
    # Get bounding box
    bb = alignment.getLargestFaceBoundingBox(img)
    
    # Align image
    img = align_image(img)
    
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    
    # obtain embedding vector for image
    features = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
    
    # Get the class for prediction
    predScores = svc.predict_proba([features])
    print(predScores)
    predClass = None
    
    maxProb = (predScores[0]).max()
    
    print(maxProb)
    
    if(maxProb >= 0.8):
    
        predClass = [(predScores[0]).argmax()]
        print(predClass)
        predClass = encoder.inverse_transform(predClass)[0]
    
    
    print(predClass)
    
    return bb, predClass, maxProb

In [30]:
students = []

cap = cv2.VideoCapture(1)
counter = 0
studentList = []
try:
    while(True):
        counter+=1
        # Capture frame-by-frame
        ret, frame = cap.read()
    #     print(ret, " ", frame)
        if frame is not None:
            if(alignment.getLargestFaceBoundingBox(frame)!=None):
                bb, predClass, maxProb = processFrame1(frame)
                print("oo: ",predClass)
                studentList.append(predClass)
                cv2.rectangle(frame,(bb.left(),bb.top()),(bb.right(),bb.bottom()),(0,0,255),2)
                label = "" if predClass is None else predClass + " " + str(maxProb)
                print("oldLabel:", label)
                label+= " "
                label+= str(maxProb)
                cv2.putText(frame,label,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
                if counter==10:
                    predClass = str(max(set(studentList), key = studentList.count)) 
                    if predClass != 'None':
                        if predClass not in students:
                              students.append(predClass)
                        cv2.putText(frame,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    counter = 0
                    studentList = []
    #             if(predClass!=None):
    #                 cv2.putText(frame,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
    #                 if predClass not in students:
    #                     students.append(predClass)
            cv2.imshow('frame',frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    #             cv2.destroyAllWindows()
except Exception as e: 
    print(e)
    cap.release()
    cv2.destroyAllWindows()

    
    
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print(students)

[[0.03567493 0.70627967 0.2580454 ]]
0.7062796698709385
None
oo:  None
oldLabel: 
[[0.02749733 0.78084277 0.19165991]]
0.7808427651999954
None
oo:  None
oldLabel: 
[[0.03711852 0.83449976 0.12838172]]
0.8344997593893596
[1]
AbhishekRanjan
oo:  AbhishekRanjan
oldLabel: AbhishekRanjan 0.8344997593893596
[[0.0146686  0.60822886 0.37710254]]
0.6082288647320016
None
oo:  None
oldLabel: 
[[0.04572142 0.62521962 0.32905897]]
0.6252196178089094
None
oo:  None
oldLabel: 
[[0.03236171 0.68524605 0.28239224]]
0.6852460481083423
None
oo:  None
oldLabel: 
[[0.03124457 0.55644777 0.41230766]]
0.5564477717358196
None
oo:  None
oldLabel: 
[[0.02650199 0.63907834 0.33441967]]
0.6390783391559693
None
oo:  None
oldLabel: 
[[0.01857582 0.81604817 0.16537601]]
0.8160481685826201
[1]
AbhishekRanjan
oo:  AbhishekRanjan
oldLabel: AbhishekRanjan 0.8160481685826201
[[0.02771815 0.82793286 0.14434899]]
0.8279328590299135
[1]
AbhishekRanjan
oo:  AbhishekRanjan
oldLabel: AbhishekRanjan 0.8279328590299135
[[0.02254

[[0.01810003 0.89099924 0.09090073]]
0.8909992364089604
[1]
AbhishekRanjan
oo:  AbhishekRanjan
oldLabel: AbhishekRanjan 0.8909992364089604
[[0.01742165 0.88938671 0.09319164]]
0.8893867069365158
[1]
AbhishekRanjan
oo:  AbhishekRanjan
oldLabel: AbhishekRanjan 0.8893867069365158
[[0.01576956 0.89802353 0.08620691]]
0.898023528283728
[1]
AbhishekRanjan
oo:  AbhishekRanjan
oldLabel: AbhishekRanjan 0.898023528283728
[[0.01840017 0.82447132 0.1571285 ]]
0.8244713219601542
[1]
AbhishekRanjan
oo:  AbhishekRanjan
oldLabel: AbhishekRanjan 0.8244713219601542
[[0.01513055 0.87638521 0.10848424]]
0.8763852111727792
[1]
AbhishekRanjan
oo:  AbhishekRanjan
oldLabel: AbhishekRanjan 0.8763852111727792
[[0.01458929 0.89185928 0.09355143]]
0.8918592786459849
[1]
AbhishekRanjan
oo:  AbhishekRanjan
oldLabel: AbhishekRanjan 0.8918592786459849
[[0.02674726 0.83736047 0.13589227]]
0.8373604701737328
[1]
AbhishekRanjan
oo:  AbhishekRanjan
oldLabel: AbhishekRanjan 0.8373604701737328
[[0.01665529 0.88636102 0.096

In [27]:
str(maxProb)

'0.7754344522220399'

In [28]:
cv2.imshow("f",frame)
cv2.waitKey(3000)
cv2.destroyAllWindows()